Copyright (c) 2023 Giovanni Squillero <giovanni.squillero@polito.it>  
`https://github.com/squillero/computational-intelligence`  
Free for personal or classroom use; see LICENSE.md for details.

# Halloween Challenge
Find the best solution with the fewest calls to the fitness functions for:
 - `num_points = [100, 1_000, 5_000]`
 - `num_sets = num_points`
 - `density = [.3, .7]`

In [217]:
from itertools import product
from random import random, randint, seed, choice
import numpy as np
from scipy import sparse
from tqdm.notebook import tqdm
from collections import namedtuple
from pprint import pprint
from typing import Tuple, Callable, Any
Evaluation = namedtuple("Evaluation", ['valid', 'cost', 'vis'])
State = Tuple[bool]

In [218]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

Defining constants and a function that allows us to estimate 


In [219]:
results = dict()

def test_solution(solve_fn: Callable[[sparse.lil_array], Any], *, name: str = None, complete: bool = False) -> None:
    if complete:
        POINTS_SET_LIST = [100, 1_000, 5_000]
        DENSITIES_LIST = [.3, .7]
    else:
        POINTS_SET_LIST = [10]
        DENSITIES_LIST = [.3]
    if name is None:
        name = solve_fn.__qualname__
    totlen = (len(POINTS_SET_LIST)**2) * len(DENSITIES_LIST)
    pbar = tqdm(enumerate(product(POINTS_SET_LIST, POINTS_SET_LIST, DENSITIES_LIST)), total=totlen, unit="problem")
    for i, (num_points, num_sets, density) in pbar:
        pbar.set_description(f"{name} - generating")
        problem = make_set_covering_problem(num_points, num_sets, density)
        pbar.set_description(f"{name} - solving {num_points}x{num_sets}/{density}")
        result = solve_fn(problem)
        resdict = results.get(name, dict())
        resdict[f"{num_points}x{num_sets}/{density}"] = result 
        results[name] = resdict

# Hill Climbing

In [224]:
def hill_climbing(sets: sparse.lil_array) -> Any:
    num_sets, problem_size = sets.shape

    def evaluate(state: State) -> Evaluation:
        vis = list(sets[state, [i]].sum() for i in range(problem_size))
        cost = -(sum(abs(np.ones(problem_size) - vis))/problem_size)
        # print(f"With {vis=} cost of {cost}")
        valid = all(list(sets[state, [i]].sum() != 0 for i in range(problem_size)))
        cost = cost if valid else -(problem_size**2)
        return Evaluation(valid, cost, vis) 

    def tweak(state: State, iter: int = 0) -> State: 
        rind = iter % num_sets
        new_state = [*state] 
        if evaluate(new_state).valid:
            new_state[rind] = False
        else:
            new_state[rind] = True
        return new_state
    
    curr_state = [choice([True, False]) for _ in range(problem_size)]
    curr_ev = evaluate(curr_state)
    pbar = tqdm(range(100), colour="blue", leave=False, unit="step")
    for step in pbar:
        new_state = tweak(curr_state, step)
        new_ev = evaluate(new_state)
        if new_ev.cost >= curr_ev.cost:
            curr_state = [*new_state]
            curr_ev = new_ev
    return curr_ev

In [227]:
test_solution(hill_climbing, name="Hill Climbing", complete=True)

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?step/s]

  0%|          | 0/100 [00:00<?, ?step/s]

  0%|          | 0/100 [00:00<?, ?step/s]

  0%|          | 0/100 [00:00<?, ?step/s]

  0%|          | 0/100 [00:00<?, ?step/s]

  0%|          | 0/100 [00:00<?, ?step/s]

IndexError: row index (103) out of bounds

In [228]:
pprint(results)

{'Hill Climbing': {'100x100/0.3': Evaluation(valid=True, cost=-1.63, vis=[3, 1, 7, 2, 2, 1, 1, 2, 1, 2, 4, 4, 1, 1, 3, 3, 4, 2, 2, 4, 2, 1, 3, 3, 3, 2, 5, 3, 3, 2, 2, 1, 1, 2, 2, 1, 3, 3, 1, 3, 2, 2, 3, 3, 1, 2, 2, 3, 3, 2, 3, 4, 2, 4, 6, 2, 3, 1, 4, 1, 2, 4, 3, 5, 3, 2, 4, 3, 4, 3, 1, 2, 3, 3, 3, 3, 4, 1, 4, 1, 4, 4, 5, 2, 2, 2, 3, 1, 3, 4, 2, 3, 3, 2, 1, 2, 4, 1, 4, 4]),
                   '100x100/0.7': Evaluation(valid=True, cost=-1.19, vis=[3, 3, 3, 3, 2, 3, 2, 2, 3, 3, 1, 3, 3, 3, 1, 2, 2, 3, 3, 3, 2, 3, 2, 2, 1, 2, 2, 3, 3, 2, 2, 1, 1, 2, 3, 1, 2, 3, 2, 3, 1, 2, 1, 1, 2, 2, 1, 2, 3, 1, 2, 3, 2, 2, 2, 2, 2, 2, 3, 1, 3, 3, 3, 1, 2, 2, 2, 3, 1, 1, 3, 2, 3, 1, 2, 2, 3, 1, 3, 3, 3, 3, 2, 2, 3, 1, 3, 2, 3, 2, 1, 3, 3, 3, 1, 1, 2, 3, 3, 1]),
                   '100x1000/0.3': Evaluation(valid=True, cost=-1.79, vis=[3, 1, 1, 3, 4, 1, 4, 1, 3, 3, 3, 4, 3, 3, 3, 5, 4, 3, 2, 4, 3, 2, 3, 1, 2, 2, 4, 3, 4, 2, 4, 3, 2, 3, 3, 3, 4, 3, 3, 2, 4, 2, 4, 5, 4, 4, 3, 2, 3, 3, 2, 2, 1, 2, 3, 4, 2, 1,

[[False False False False False  True False  True]
 [ True  True False False False  True  True  True]
 [False False False  True False False  True False]
 [False False  True False False  True  True False]
 [ True False False False False False  True False]
 [ True  True False  True  True False False  True]
 [ True False  True False  True False False False]
 [False  True False False  True False False  True]]
[1, 1, 1, 1, 0, 3, 3, 2]
